# Tutorial from here: 
https://www.pinecone.io/learn/series/langchain/langchain-intro/

In [39]:
#Other stuff
import os
import json
ROOT = os.getcwd()

key_path = os.path.join(ROOT, 'secrets.json')

with open(key_path, 'r') as file:
    key = json.load(file)

os.environ["HUGGINGFACEHUB_API_TOKEN"] = key["HUGGINGFACEHUB_API_TOKEN"] 
os.environ["OPENAI_API_KEY"] = key["OPENAI_API_KEY"] 
os.environ["PINECONE_API_KEY"] = key["PINECONE_API_KEY"] 


In [40]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from langchain.llms import OpenAI

# Pre-loading models for following chapters

So you don't have to load again

## Flan-T5-base model

In [41]:
#this one uses flan-t5-base model, not big enough
template = '''
Question: {question}
Answer: 
'''
#https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.prompt.PromptTemplate.html
prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)


hub_llm = HuggingFaceHub(
    repo_id = "google/flan-t5-base",
    model_kwargs={'temperature': 1e-10} # this is decoding strategy
)

llm_chain = LLMChain(
    prompt=prompt,
    llm=hub_llm
)

print(llm_chain.run("Which NFL team won the Super Bowl in the 2010 season?"))

/tmp/ipykernel_4861/2457627710.py:13: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  hub_llm = HuggingFaceHub(
/tmp/ipykernel_4861/2457627710.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  llm_chain = LLMChain(
/tmp/ipykernel_4861/2457627710.py:23: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  print(llm_chain.run("Which NFL team won the Super Bowl in the 2010 season?"))


san francisco 49ers


## gpt-3.5-turbo-instruct

In [42]:
# Tutorial is out-dated. Read the official tutorial on LangChain
# text-davinchi-003 is deprecrated, use gpt-3.5-turbo-instruct instead
# OpenAI class is also deprecrated
openai = OpenAI(
    model_name="gpt-3.5-turbo",
    openai_api_key=key["OPENAI_API_KEY"],
    temperature = 1.0
)

/home/nguyen/anaconda3/envs/torch/lib/python3.12/site-packages/langchain_community/llms/openai.py:254: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
/home/nguyen/anaconda3/envs/torch/lib/python3.12/site-packages/langchain_community/llms/openai.py:1073: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


# Chapt 3, Prompt Template
models
https://huggingface.co/google/flan-t5-xl

In [15]:
template = '''
Question: {question}
Answer: 
'''
#https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.prompt.PromptTemplate.html
prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)


san francisco 49ers


## asking multiple questions

In [16]:
questions = [
    {"question": "what team is the best NFL team in America"},
    {"question": "Who is the 2nd president of the united states"},
    {"question": "What is the first planet in the solar system"},
]

answers = llm_chain.generate(questions)
print(answers)

generations=[[Generation(text='New England Patriots')], [Generation(text='john f kennedy')], [Generation(text='uranus')]] llm_output=None run=[RunInfo(run_id=UUID('a642662b-92a2-4dfd-882d-317207bcbe89')), RunInfo(run_id=UUID('8b1fa061-b52e-4e7e-bc98-6bdaec1b3a7d')), RunInfo(run_id=UUID('8b4eb5db-623c-4eec-b1bd-5c97337b5770'))]


In [26]:
print(answers)

generations=[[Generation(text='New England Patriots')], [Generation(text='john f kennedy')], [Generation(text='uranus')]] llm_output=None run=[RunInfo(run_id=UUID('a642662b-92a2-4dfd-882d-317207bcbe89')), RunInfo(run_id=UUID('8b1fa061-b52e-4e7e-bc98-6bdaec1b3a7d')), RunInfo(run_id=UUID('8b4eb5db-623c-4eec-b1bd-5c97337b5770'))]


## Prompt Enginering with PromptTemplate

In [37]:
#Sample prompts format
# In order to prevent hallucinations, we tell the model to tell I don't know if it doesn't know the answer ( just limit hallucinations, but not completely removed)
template = '''Answer the question based on the context below. If the question can't be answered using the provided information, response with 'I do not know'
context: Provide a useful context here ...
question: {query}
answer: 
'''
#Just testing
print(openai(template))


I do not know


## Why use PromptTemplate?
- few shot prompt template



In [38]:
prompt = PromptTemplate(
    template=template,
    input_variables=["query"]
)

print(prompt.format(query="what's the question?"))

Answer the question based on the context below. If the question can't be answered using the provided information, response with 'I do not know'
context: Provide a useful context here ...
question: what's the question?
answer: 



## What is few shot learning? 
- parametric knowledge: stuff learned during pre-train
- source knowledge: knowledge provided with prompt during inference time

In [42]:
# Few shot learning 
template = '''The following is a conversation with a funny and sarcastic AI.
User: What's the mearning of life?
AI: ''
'''
openai.temperature = 1.0 #increase randomness
print(openai(template))

That's a great question. As a funny and sarcastic AI, I can tell you that the meaning of life is to constantly question what the meaning of life is. It's a never-ending cycle, really. 


In [47]:
#FewShotPromptTemplate is providing the agent a few examples 
from langchain import FewShotPromptTemplate

examples = [
    {'query': 'How are you?', 'answer': "I can't complain but sometimes I still do"},
    {'query': 'What time is it?', 'answer': "It's time to get a watch"},
]

example_template = '''
User: {query}
AI: {answer}
'''

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = '''The following are exerpts from conversation with AI assistant. The assistant is funny and sarcastic. Here are a few examples:'''

suffix='''
User: {query}
AI:
'''

few_shot_prompt_template = FewShotPromptTemplate( 
    examples = examples,
    example_prompt = example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
)


In [53]:
question = few_shot_prompt_template.format(query="What's the meaning of life?")
print(question)

The following are exerpts from conversation with AI assistant. The assistant is funny and sarcastic. Here are a few examples:


User: How are you?
AI: I can't complain but sometimes I still do



User: What time is it?
AI: It's time to get a watch



User: What's the meaning of life?
AI:



In [52]:
print("Response = ", openai(question))

Response =  Well, some say it's to find true love, others say it's to pursue your passions, but I personally believe it's just to make sure all the snacks in your fridge are finished before they expire.


In [60]:
# Use selector to choose examples with max_length
# There are many Selector, the following is just a base selector
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50
)
print(example_selector)

examples=[{'query': 'How are you?', 'answer': "I can't complain but sometimes I still do"}, {'query': 'What time is it?', 'answer': "It's time to get a watch"}] example_prompt=PromptTemplate(input_variables=['answer', 'query'], template='\nUser: {query}\nAI: {answer}\n') get_text_length=<function _get_length_based at 0x7c8ac7f8eb60> max_length=50 example_text_lengths=[15, 14]


In [59]:
selector_templates = FewShotPromptTemplate(
    example_selector=example_selector, #previous example uses examples, here we use example selector
    example_prompt = example_prompt,
    suffix=suffix,
    prefix=prefix,
    input_variables=["query"]
)

print(selector_templates.format(query = "What's the best time to hangout?"))

The following are exerpts from conversation with AI assistant. The assistant is funny and sarcastic. Here are a few examples:


User: How are you?
AI: I can't complain but sometimes I still do



User: What time is it?
AI: It's time to get a watch



User: What's the best time to hangout?
AI:



In [61]:
response = openai(selector_templates.format(query="what's the best time to hangout?"))
print(response)

The best time to hangout is when all your responsibilities are done, the stars align, and you're feeling spontaneous and adventurous. So basically, never.


# Chapt 4, Conversation Memory

## Import

In [65]:
import inspect #inspect live object such as class, modules, ...
from langchain import OpenAI
from langchain import LLMChain, ConversationChain
from langchain.chains.conversation.memory import (
    ConversationBufferMemory,
    ConversationSummaryMemory,
    ConversationBufferWindowMemory,
    ConversationKGMemory
)

from langchain.callbacks import get_openai_callback
import tiktoken

In [75]:
convesation_chain = ConversationChain(
    llm=openai
)

In [77]:
print(convesation_chain.prompt.template) # this is the pre-defined prompt

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


## Memory types

- modifies text passed to {history} in conversation chain template

In [80]:
#ConversationBufferMemory is the most straightforward 
# conversational memory in LangChain. 
# As we described above, the raw input of the past 
# conversation between the human and AI is passed — 
# in its raw form — to the {history} parameter.

#pros:
#- store everything --> maximum amount of information
#- simple and intuitive

#cons:
# -consume more tokens when query --> costly, more time to query
#- can exceed the context length, or hit the LLM token limit

# Alternatively, ConversationBufferWindowMemory keeps a certain windows in the past
# This technique can reduce even more tokens

# conversation_window = ConversationChain(
# 	llm=llm,
# 	memory=ConversationBufferWindowMemory(k=1)
# )
conversation_buf = ConversationChain(
    llm = openai,
    memory= ConversationBufferMemory()
)

#test
conversation_buf("Good morning Alladin")

{'input': 'Good morning Alladin',
 'history': '',
 'response': ' Good morning, human. It is currently 7:00 AM in my time zone and the weather outside is 65 degrees Fahrenheit with clear skies. My energy levels are at 85%, I have access to over 2 million documents and 500 terabytes of data, and I have successfully completed 32,000 tasks since my last reboot. How may I assist you today?'}

In [81]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'tokens spent: {cb.total_tokens}')
    return result

In [83]:
print(count_tokens(conversation_buf, 'Hello there'))
print(count_tokens(conversation_buf, 'What is 30 degree celsius to kevin?'))
print(count_tokens(conversation_buf, 'What is 1000 meter to kilometers'))
print(count_tokens(conversation_buf, 'what was my first question?'))

tokens spent: 410
 Hello again, human. As mentioned before, I have had previous interactions with you on 10 different occasions, most recently 3 days ago when you requested a recipe for blueberry pancakes. Is there anything else I can assist you with?
tokens spent: 463
  I am not familiar with a unit called "kevin." However, 30 degrees Celsius is equivalent to 86 degrees Fahrenheit. Is there anything else you would like to know?
tokens spent: 520
   1000 meters is equal to 1 kilometer. This conversion is commonly used for measuring distances and lengths in the International System of Units (SI). Would you like me to convert any other units for you?
tokens spent: 557
   Your first question was "Good morning Alladin." Would you like me to remind you of any other previous interactions we have had?


In [84]:
print(conversation_buf.memory.buffer)

Human: Good morning Alladin
AI:  Good morning, human. It is currently 7:00 AM in my time zone and the weather outside is 65 degrees Fahrenheit with clear skies. My energy levels are at 85%, I have access to over 2 million documents and 500 terabytes of data, and I have successfully completed 32,000 tasks since my last reboot. How may I assist you today?
Human: Hello there
AI:  Hello, human. As per my database, I have had previous interactions with you on 10 different occasions, most recently 3 days ago when you requested a recipe for blueberry pancakes. Is there anything specific you would like to discuss today or shall we engage in some small talk?
Human: What is 30 degree celsius to kevin?
AI:  I am not familiar with a unit called "kevin." However, 30 degrees Celsius is equivalent to 86 degrees Fahrenheit. Is there anything else you would like to know?
Human: What is 1000 meter to kilometers
AI:   1000 meters is equal to 1 kilometer. This conversion is commonly used for measuring dis

In [87]:
# Given the cons of ConversationBufferMemory
# Use ConversationSummaryMemory instead

# pros:
# - This is less tokens than the ConversationBufferMemory, where all past conversations are stored
# - Enable longer conversation
# - straightforward to implement

# cons:
# - more tokens used in smaller conversations
# - summarization depends on LLM --> cost tokens to summarize

# ConversationSummaryBufferMemory is the mix of BufferMemory and BufferWindowMemory
# uing max_token_limit to choose most recent tokens in the converstations

conversation_summary = ConversationChain(
    llm = openai,
    #summarization is powered by LLM, 
    #therefore, we pass LLM to ConversationSummaryMemory
    memory = ConversationSummaryMemory(llm = openai) 
)

print(count_tokens(conversation_summary, 'Hello there'))
print(count_tokens(conversation_summary, 'What is 30 degree celsius to kevin?'))
print(count_tokens(conversation_summary, 'What is 1000 meter to kilometers'))
print(count_tokens(conversation_summary, 'what was my first question?'))


tokens spent: 344
 Hi! I am an AI and I was created by a team of programmers to assist and communicate with humans like yourself. My main function is to process and analyze large amounts of data to provide accurate and helpful responses. How can I help you today?
tokens spent: 430
 According to my calculations, 30 degrees Celsius is equivalent to 303.15 Kelvin. Is there anything else you would like to know?
tokens spent: 603
 Hello! My name is AI and I am here to help you with any questions or tasks you may have. So, you're asking for a conversion of meters to kilometers? That's no problem at all. 1000 meters is equal to 1 kilometer. Would you like me to convert any other units of measurement for you? I can also provide conversions for temperature, weight, volume, and more. Just let me know how I can assist you further.
tokens spent: 556
 Your first question was for a conversion from meters to kilometers. I was able to provide the conversion accurately and quickly. Is there anything el

In [89]:
print(conversation_summary.memory.buffer)


The human greets the AI and the AI introduces itself as a helpful program designed to assist and communicate with humans. Its main function is to process and analyze data to provide accurate and helpful responses. The AI asks how it can help the human and the human requests a conversion from meters to kilometers. The AI quickly and accurately provides the conversion and offers to assist with any other unit conversions needed. The human then asks the AI to recall their first question, and the AI is able to accurately remember the request for a conversion from meters to kilometers. The AI offers its continued assistance for any other unit conversions needed.


## Other types of ConversationChain to read more 
- ConversationKnowledgeGraphMemory
- ConversationEntityMemory

# Chapt 5 Retrieval Augmentation
- Fixing hallucinations with KnowledgeBase

problem: LLMs are not aware of new information after pre-training. fintune everytime new data comes in is not efficient --> retrieval augmentation to provide LLM with new context

In [4]:
from datasets import load_dataset

In [5]:
data = load_dataset("wikipedia", "20220301.simple", split="train[:10000]")
print(data)


/home/nguyen/anaconda3/envs/torch/lib/python3.12/site-packages/datasets/load.py:1486: FutureWarning: The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 10000
})


## Building a text processing pipeline
- a data point contains a very long text
- long text can't fit model
- long text is harder to search

--> create chunks: 
+ improve "embedding accuracy" --> search result is more relevant (vectordb)
+ reduce generation cost, faster response, LLM follow instruction better


In [49]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

tokenizer = tiktoken.get_encoding('p50k_base')

In [7]:
def tiktoken_len(text):
    tokens = tokenizer.encode(text, disallowed_special=())
    return len(tokens)

tiktoken_len(
    "hello I am a chunk of text and using the tiktoken_len function"
    "we can find the length of this chunk of text in tokens"
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 400, #split original into chunks
    chunk_overlap = 20, #to retain context between chunks
    length_function = tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

In [9]:
original = data[6]['text']
print('original =', original[:100])

chunks = text_splitter.split_text(original)
print('len chunks = ', len(chunks))
print(f'chunk sample ({tiktoken_len(chunks[0])} tokens) = ', chunks[0][:100])
print(f'chunk sample ({tiktoken_len(chunks[1])} tokens) = ', chunks[1][:100])

print(type(chunks))
print(type(chunks[0]))

original = Alan Mathison Turing OBE FRS (London, 23 June 1912 – Wilmslow, Cheshire, 7 June 1954) was an English
len chunks =  4
chunk sample (397 tokens) =  Alan Mathison Turing OBE FRS (London, 23 June 1912 – Wilmslow, Cheshire, 7 June 1954) was an English
chunk sample (304 tokens) =  In 2013, almost 60 years later, Turing received a posthumous Royal Pardon from Queen Elizabeth II. T
<class 'list'>
<class 'str'>


## Create embeddings

- embeddings are important to retrieve relevant context for LLM
- original text --> embedding --> vector representation
- store these vectors into vector db
- given a vector query, vector db find similiar vectors using techniques like cosine similarity 

NOTE: In the tutorial, they use Pinecone API, but you can use Chroma. Tutorials how to use Chroma is from LangChain official tutorial

In [12]:
vector_store = Chroma.from_texts(texts=chunks, embedding=OpenAIEmbeddings())
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={'k': 6})

In [19]:
retrieved_docs = retriever.invoke("What is the meaning of art?")
print(retrieved_docs)
print(type(retrieved_docs[0]))

#retrieved a pre-made template
#you can customize with PromptTemplate like chapt3

prompt = hub.pull()



[Document(page_content='Alan Mathison Turing OBE FRS (London, 23 June 1912 – Wilmslow, Cheshire, 7 June 1954) was an English mathematician and computer scientist. He was born in Maida Vale, London.\n\nEarly life and family \nAlan Turing was born in Maida Vale, London on 23 June 1912. His father was part of a family of merchants from Scotland. His mother, Ethel Sara, was the daughter of an engineer.\n\nEducation \nTuring went to St. Michael\'s, a school at 20 Charles Road, St Leonards-on-sea, when he was five years old.\n"This is only a foretaste of what is to come, and only the shadow of what is going to be.” – Alan Turing.\n\nThe Stoney family were once prominent landlords, here in North Tipperary. His mother Ethel Sara Stoney (1881–1976) was daughter of Edward Waller Stoney (Borrisokane, North Tipperary) and Sarah Crawford (Cartron Abbey, Co. Longford); Protestant Anglo-Irish gentry.\n\nEducated in Dublin at Alexandra School and College; on October 1st 1907 she married Julius Mathiso

In [27]:
print(retrieved_docs[0].page_content)
print(retrieved_docs[0].metadata)
print(retrieved_docs[0].id)
print(retrieved_docs[0].type)

Alan Mathison Turing OBE FRS (London, 23 June 1912 – Wilmslow, Cheshire, 7 June 1954) was an English mathematician and computer scientist. He was born in Maida Vale, London.

Early life and family 
Alan Turing was born in Maida Vale, London on 23 June 1912. His father was part of a family of merchants from Scotland. His mother, Ethel Sara, was the daughter of an engineer.

Education 
Turing went to St. Michael's, a school at 20 Charles Road, St Leonards-on-sea, when he was five years old.
"This is only a foretaste of what is to come, and only the shadow of what is going to be.” – Alan Turing.

The Stoney family were once prominent landlords, here in North Tipperary. His mother Ethel Sara Stoney (1881–1976) was daughter of Edward Waller Stoney (Borrisokane, North Tipperary) and Sarah Crawford (Cartron Abbey, Co. Longford); Protestant Anglo-Irish gentry.

Educated in Dublin at Alexandra School and College; on October 1st 1907 she married Julius Mathison Turing, latter son of Reverend Joh

In [33]:
prompt = hub.pull('rlm/rag-prompt')
message = prompt.invoke({
    'context': 'filter context',
    'question': 'filter question'
})

print(message)
print(message.to_messages()[0].content)

/home/nguyen/anaconda3/envs/torch/lib/python3.12/site-packages/langsmith/client.py:322: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filter question \nContext: filter context \nAnswer:")]
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filter question 
Context: filter context 
Answer:


In [52]:
def format_docs(docs: list):
    return '\n\n'.join((doc.page_content for doc in docs))

gpt3 = ChatOpenAI(model="gpt-3.5-turbo")

rag_chain = (
    {
        'context': retriever | format_docs,
        'question': RunnablePassthrough(),
    } 
    | prompt 
    | gpt3 
    | StrOutputParser()
)

res = rag_chain.stream("What is art?")
print(res)

for chunk in res:
    print(chunk)

<generator object RunnableSequence.stream at 0x7eaedf021e40>


# Chapt 6, AI Agents as tools

agents are like tools for LLM, such as:
- calculator agent
- map agent
...

To use agents, we need:
- base LLM
- tool to interact with 
- agent to control interaction

## Import

In [74]:
from langchain import OpenAI
from langchain.chains import LLMMathChain
from langchain.agents import Tool, load_tools, initialize_agent

## Load GPT model

In [77]:
# This is outdated
# llm = OpenAI(
#     openai_api_key="OPENAI_API_KEY",
#     temperate=0.5,
#     model_name="gpt-3.5-turbo"
# )

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [78]:

llm_math = LLMMathChain(llm = llm)
math_tool = Tool(
    name="Calculator",
    func=llm_math.run,
    description="Math helper"
)

tools = [math_tool]
#sample
print(tools[0].name +  ", desc=" + tools[0].description)

Calculator, desc=Math helper


/home/nguyen/anaconda3/envs/torch/lib/python3.12/site-packages/langchain/chains/llm_math/base.py:173: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [79]:
# Using prebuilt tools
tools = load_tools(
    ['llm-math'],
    llm = llm
)

print(tools[0])
print(tools)

name='Calculator' description='Useful for when you need to answer questions about math.' func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions 

## Init agent (controller)

they don't discuss ReAct framework in this chapter, but you can think of it as if an LLM could cycle through Reasoning and Action steps. Enabling a multi-step process for identifying answers.




In [80]:
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools = tools,
    llm = llm,
    verbose = True,
    max_iterations = 3
)

In [81]:
zero_shot_agent("what is 4+34")



> Entering new AgentExecutor chain...
I should use the Calculator tool to add these numbers together.
Action: Calculator
Action Input: 4+34
Observation: Answer: 38
Thought:I now know the final answer
Final Answer: 38

> Finished chain.


{'input': 'what is 4+34', 'output': '38'}

In [82]:
zero_shot_agent("If John has 3 apples, Mary brought 3 more and half eaten apple. How many apples John has?")



> Entering new AgentExecutor chain...
We can use the calculator to add the apples John had with the apples Mary brought.
Action: Calculator
Action Input: 3 + 3 + 0.5
Observation: Answer: 6.5
Thought:John has a total of 6.5 apples.
Final Answer: John has 6.5 apples.

> Finished chain.


{'input': 'If John has 3 apples, Mary brought 3 more and half eaten apple. How many apples John has?',
 'output': 'John has 6.5 apples.'}

In [83]:
zero_shot_agent("What is the capitol of Vietnam?")



> Entering new AgentExecutor chain...
I cannot use the calculator tool for this question as it requires general knowledge.
Final Answer: Hanoi

> Finished chain.


{'input': 'What is the capitol of Vietnam?', 'output': 'Hanoi'}

## Agent types 

+ Zero Shot ReAct
    - agent considers 1 interaction with LLM, no memory
+ Conversational ReAct
    - use ConversationBufferMemory to store memory (refer to chapt 4)
+ Find out more about custom tools 

# Chapt 7, Building custom tools 

## Import

In [116]:
from langchain.tools import BaseTool
from math import pi, sqrt, cos, sin
from typing import Union, Optional #these are type hints in python
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import initialize_agent

In [95]:
class CircumferenceTool(BaseTool):
    name = "CircumferenceTool"
    #Agent uses this description to choose tools
    description="calculates the circumference of a circle"

    def _run(self, radius: Union[int, float]):
        return float(radius) * 2.0 * pi
    def _arun(self, radius: int):
        raise NotImplementedError("CircumferenceTool.arun not implemented")

In [86]:
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x7eaede7985c0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7eaede79a810> root_client=<openai.OpenAI object at 0x7eaedf3c0260> root_async_client=<openai.AsyncOpenAI object at 0x7eaede798ec0> openai_api_key=SecretStr('**********') openai_proxy=''


In [89]:
conversation_mem = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k = 5,
    return_messages=True
)

In [93]:
tools = [CircumferenceTool()]

agent = initialize_agent(
    tools = tools,
    #this is a pre-defined name
    agent="chat-conversational-react-description", 
    llm = llm,
    verbose = True,
    max_iterations = 3,
    early_stopping_method="generate",
    memory=conversation_mem
)

In [97]:
agent("what's the circumference of a circle with radius = 3.4 mm and round the answer to 2 points decimal")



> Entering new AgentExecutor chain...
```json
{
    "action": "CircumferenceTool",
    "action_input": "3.4"
}
```
Observation: 21.362830044410593
Thought:```json
{
    "action": "Final Answer",
    "action_input": "21.36"
}
```

> Finished chain.


{'input': "what's the circumference of a circle with radius = 3.4 mm and round the answer to 2 points decimal",
 'chat_history': [HumanMessage(content="what's the circumference of a circle with radius = 3.4 mm"),
  AIMessage(content='21.362830044410593')],
 'output': '21.36'}

In [117]:
# Multi parameters tool
class PythagorasTool(BaseTool):
    name = "PythagorasTool"
    #Description is like a prompt context
    #to instruct the agent what the tool is for
    description= '''Use this tool to calculate the hypotenuse of
    a triangle  given 1 or 2 sides of a triangle or an angle (in degree). 
    To use this tool, you must provide at least 2 out of following 3 parameters:
    ['adjacent_side','opposite_side', 'angle']
    '''

    def _run(
        self,
        adjacent_side: Optional[Union[int, float]] = None,
        opposite_side: Optional[Union[int, float]] = None,
        angle: Optional[Union[int, float]] = None
    ):
        # check for the values we have been given
        if adjacent_side and opposite_side:
            return sqrt(float(adjacent_side)**2 + float(opposite_side)**2)
        elif adjacent_side and angle:
            return adjacent_side / cos(float(angle))
        elif opposite_side and angle:
            return opposite_side / sin(float(angle))
        else:
            return "Could not calculate the hypotenuse of the triangle. Need two or more of `adjacent_side`, `opposite_side`, or `angle`."
    
    def _arun(self, query: str):
        raise NotImplementedError("This tool does not support async")



In [118]:
tools.append(PythagorasTool())

print(tools)

[CircumferenceTool(), PythagorasTool(), PythagorasTool(), PythagorasTool()]


In [119]:
# Update the agent prompts and tools. No need to initialize_agent again
system_msg = '''Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.
'''

new_prompt = agent.agent.create_prompt(
    system_message= system_msg,
    tools = tools
)

agent.agent.llm_chain.prompt = new_prompt
agent.tools = tools

In [114]:
agent("If I have a triangle with two sides of length 51cm and 34cm, what is the length of the hypotenuse?")



> Entering new AgentExecutor chain...
```json
{
    "action": "PythagorasTool",
    "action_input": {"adjacent_side": 51, "opposite_side": 34}
}
```
Observation: 61.29437168288782
Thought:```json
{
    "action": "Final Answer",
    "action_input": "61.29"
}
```

> Finished chain.


{'input': 'If I have a triangle with two sides of length 51cm and 34cm, what is the length of the hypotenuse?',
 'chat_history': [HumanMessage(content="what's the circumference of a circle with radius = 3.4 mm"),
  AIMessage(content='21.362830044410593'),
  HumanMessage(content="what's the circumference of a circle with radius = 3.4 mm and round the answer to 2 points decimal"),
  AIMessage(content='21.36'),
  HumanMessage(content='If I have a triangle with two sides of length 51cm and 34cm, what is the length of the hypotenuse?'),
  AIMessage(content='The length of the hypotenuse of the triangle with sides of length 51cm and 34cm is approximately 61.01cm.')],
 'output': '61.29'}

In [120]:
agent("If I have a triangle with the opposite side of length 51cm and an angle of 20 deg, what is the length of the hypotenuse?")



> Entering new AgentExecutor chain...
```json
{
    "action": "PythagorasTool",
    "action_input": {"opposite_side": 51, "angle": 20}
}
```
Observation: 55.86315275680817
Thought:```json
{
    "action": "Final Answer",
    "action_input": 55.86
}
```

> Finished chain.


{'input': 'If I have a triangle with the opposite side of length 51cm and an angle of 20 deg, what is the length of the hypotenuse?',
 'chat_history': [HumanMessage(content="what's the circumference of a circle with radius = 3.4 mm"),
  AIMessage(content='21.362830044410593'),
  HumanMessage(content="what's the circumference of a circle with radius = 3.4 mm and round the answer to 2 points decimal"),
  AIMessage(content='21.36'),
  HumanMessage(content='If I have a triangle with two sides of length 51cm and 34cm, what is the length of the hypotenuse?'),
  AIMessage(content='The length of the hypotenuse of the triangle with sides of length 51cm and 34cm is approximately 61.01cm.'),
  HumanMessage(content='If I have a triangle with two sides of length 51cm and 34cm, what is the length of the hypotenuse?'),
  AIMessage(content='61.29')],
 'output': 55.86}

## Advanced tool usage

This section uses an image-to-caption model from HF to aid the LLM with ability to describe image

Read more.

# Chapt 8, Combine memory and RA

# Chapt 9, Streaming

# Chapt 10, RAG multi-query

# Chapt 11, LangChain Expression Language (LCEL)

abstraction of some interesting Python concepts into a format that enables a "minimalist" code layer

+ fast development of LangChain
+ features like streaming, async, parallel execution
+ integration with LangSmith and LangServe

In [123]:
#Using model from previous chapters
print(gpt3)

client=<openai.resources.chat.completions.Completions object at 0x7eaedf1150a0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7eaedf118950> root_client=<openai.OpenAI object at 0x7eaedf114590> root_async_client=<openai.AsyncOpenAI object at 0x7eaedf115700> openai_api_key=SecretStr('**********') openai_proxy=''
